In [17]:
import os
import re
import string
import random
import time
from tqdm import tqdm
import numpy as np
import pandas as pd
import torch
from torch import nn

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split


import matplotlib.pyplot as plt
from transformers import BertTokenizer
from transformers import BertModel
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

import sys
sys.path.append("../../..")
from utility import parse_config, seed_everything, custom_print
from preprocess_class import create_dataset

In [16]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
#Trainer arguments

# Define the training loop
def train(model_name, train_dataset, eval_dataset):
    # Load the model
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    # Define the training arguments
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=3,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=64,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
        evaluation_strategy='steps',
        eval_steps=50,
        save_total_limit=5,
    )

    # Define the optimizer and scheduler
    optimizer = AdamW(model.parameters(), lr=5e-5)
    num_training_steps = len(train_dataset) // training_args.per_device_train_batch_size * training_args.num_train_epochs
    lr_scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=training_args.warmup_steps, num_training_steps=num_training_steps)

    # Define the trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        optimizer=optimizer,
        scheduler=lr_scheduler,
        compute_metrics=compute_metrics,
        device = device
    )

    # Train the model
    trainer.train()

    return model

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/gjj980/miniforge3/envs/voc_project/lib/python3.8/site-packages/transformers/utils/import_utils.py", line 1110, in _get_module
    return importlib.import_module("." + module_name, self.__name__)
  File "/Users/gjj980/miniforge3/envs/voc_project/lib/python3.8/importlib/__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1014, in _gcd_import
  File "<frozen importlib._bootstrap>", line 991, in _find_and_load
  File "<frozen importlib._bootstrap>", line 975, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 671, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 843, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/Users/gjj980/miniforge3/envs/voc_project/lib/python3.8/site-packages/transformers/trainer.py", line 168, in <module>
    import dataset

In [ ]:
#Creating BERTClassifier class